# Análisis y preprocesamiento de los datos

Se explorarán, en pequeños experimentos, distintas formas de representación de los datos del corpus WiNER (Ghaddar y Langlais 2017) para utilizarlos en la tarea de reconocimiento de entidades nombradas. Para esto se exploran distintas combinaciones de vectores de palabras como representación de una instancia de entrenamiento (Iacobacci et al. 2016).

### Descripción del Corpus WiNER

* Documents.tar.bz2 : este archivo contiene 3256489 (TODO: check this!) artículos de Wikipedia repartidos en 3225 archivos. Cada archivo contiene aproximadamente 1000 artículos nombrados por sus respectivos IDs. Los artículos están indexados por su wikiID seguidos de oraciones (una por línea), donde las palabras son remplazadas por sus ids.

<br>
* document.vocab : este archivo contiene el mapeo de palabras (case sensitive); el formato es: 
           palabra <espacio> #ocurrencias
  El ID de cada palabra es el número de línea en la cual ocurre

In [47]:
import pandas as pd
import numpy as np
import time
import gensim
from gensim.models import Word2Vec, KeyedVectors

### Cargamos los datos

In [48]:
word_mapping = pd.read_csv('./corpus_WiNER/document.vocab', sep=' ', header=None,
                           keep_default_na=False) # con esto evito que el parser trate como 
                                                  # nan value a algunas palabras.
word_mapping.columns = ['word', 'frequency']

In [49]:
print(word_mapping.shape)
word_mapping.head()

(6931358, 2)


,word,frequency
0,the,73411953
1,",",68044881
2,.,54241850
3,of,40550466
4,and,34602894


In [50]:
len(word_mapping['word'].unique())

6931358

### Cargamos los artículos del Documento "0"

In [51]:
doc_0 = pd.read_csv('./corpus_WiNER/Documents/0', sep='ID', engine='python', header=None, names=['sentence', 'art_ID'])

In [52]:
doc_0.shape

(130209, 2)

In [53]:
doc_0.head()

,sentence,art_ID
0,NaN,431388.0
1,650 5590 753942 12 189 243 1 2039 47 257 682 1...,NaN
2,34 23207 1523 4 57193 15 10777 14353 4 157019 ...,NaN
3,418 0 1858 101 0 1322 1 5590 18 860 729 14 92 ...,NaN
4,650 5590 1523 8 1136 15 11301 575 1156 1 2746 ...,NaN


Vamos a asociarle a cada oración el ID del artículo al cual pertenece.

Es importante recordar que el orden de las oraciones está dado por los índices del dataframe.

In [54]:
art_ID_list = doc_0['art_ID'].tolist()
art_ID = 0
for idx, elem in enumerate(art_ID_list):
    if not np.isnan(elem):
        art_ID = elem
    else:
        art_ID_list[idx] = art_ID
doc_0['art_ID'] = art_ID_list

Removemos ahora las filas con NaN que contenian los ID de los artículos inicialmente.

In [55]:
doc_0 = doc_0.dropna()

In [56]:
print('El documento contiene {} oraciones.'.format(doc_0.shape[0]))
print('El documento contiene {} artículos'.format(len(doc_0['art_ID'].unique())))

El documento contiene 128395 oraciones.
El documento contiene 1814 artículos


Hacemos que cada sentencia sea una lista de palabras codificadas

In [57]:
doc_0['sentence'] = doc_0['sentence'].map(lambda x: x.split(' '))

In [58]:
doc_0.head()

,sentence,art_ID
1,"[650, 5590, 753942, 12, 189, 243, 1, 2039, 47,...",431388.0
2,"[34, 23207, 1523, 4, 57193, 15, 10777, 14353, ...",431388.0
3,"[418, 0, 1858, 101, 0, 1322, 1, 5590, 18, 860,...",431388.0
4,"[650, 5590, 1523, 8, 1136, 15, 11301, 575, 115...",431388.0
5,"[152, 642, 16, 8143, 23122, 20, 0, 662955, 16,...",431388.0


Nos quedamos con un artículo

In [59]:
article = doc_0[doc_0.art_ID == 1000]
article.head()

,sentence,art_ID
2756,"[63903, 24730, 9, 7, 3035, 4104, 7584, 1, 355,...",1000.0
2757,"[3654, 16, 2502, 39413, 1, 24730, 9, 44, 3, 99...",1000.0
2758,"[24730, 35, 49, 3521, 15, 575, 1, 15, 2440, 1,...",1000.0
2759,"[152, 112, 8, 2037, 20, 52, 54, 3035, 17572, 3...",1000.0
2760,"[64, 62, 5787, 1132, 15, 0, 144, 24730, 1312, ...",1000.0


Reconstruimos la primera oración del artículo

In [60]:
def sentence_decoder(sentence, word_mapping):
    dec_sentence = []
    for idx in sentence:
        mapped_w = word_mapping.loc[np.int(idx), 'word']
        dec_sentence.append(mapped_w)

    return dec_sentence 

In [61]:
sentence = article.loc[2756, 'sentence']

In [62]:
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

### Word embeddings utilizando el modelo pre-entrenado word2vec de Google

Utilizaremos la librería Gensim https://radimrehurek.com/gensim/

Cargamos Google's pre-trained Word2Vec model.

Utilizando KeyedVectors para cargar el modelo tiene la desventaja de que no se puede seguir entrenando. Pero es más eficiente que utilizar gensim.models.Word2Vec
https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors

In [63]:
start = time.time()
# model = KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)
# model.save('./models/word2vecGoogle.model')
model = KeyedVectors.load('./models/word2vecGoogle.model')
end = time.time()
print('demora: {}'.format(end-start))
# model = Word2Vec.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

demora: 14.532685995101929


In [64]:
print('Cantidad de word embeddings: {}'.format(len(model.vectors)))

Cantidad de word embeddings: 3000000


In [65]:
print('Dimensionalidad de los vectores: {}'.format(model.vector_size))

Dimensionalidad de los vectores: 300


## Exploremos distintas combinaciones de vectores de palabras

### Concatenación

Este método consiste en concatenar los vectores de palabras que rodean una palabra objetivo en un vector más grande, que tiene un tamaño igual a las dimensiones agregadas de todos las proyecciones (embeddings) individuales. (DUDA: tamaño = $2 W D$)?

- $w_{ij}$ = peso asociado con la i-ésima dimensión del vector de la j-ésima palabra en la oración. NOTA: con los vectores de palabras de una oración se forma una matriz $w^{\space D\space x\space L}$ donde $L$ es la cantidad de palabras de esa oración.
- $D$ = dimensionalidad de los word vectors originales. Por ejemplo, al usar el modelo word2vec de Google sabemos que D = 300.
- $W$ = tamaño de ventana que se define como el número de palabras en un solo lado.

Nos interesa representar el contexto de la I-ésima palabra de la oración. 

La i-ésima dimensión del vector de concatenación, que tiene un tamaño de $2 W D$, se calcula de la siguiente manera:

$$ e_{i} =\begin{cases} 
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space \left\lfloor{\frac{i}{D}}\right\rfloor} & \left\lfloor{\frac{i}{D}}\right\rfloor < W \\
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space 1\space  +\space\left\lfloor{\frac{i}{D}}\right\rfloor} & c.c.
   \end{cases}$$

DUDA: Cómo manejar caso de las palabras que están al comienzo (donde $W$ > #palabras a la izquierda) y al final (donde $W$ > #palabras a la derecha)?

Utilizamos padding? En tal caso, de que manera? 

- Replicar las palabras que si se pudieron agregar al vector de contexto es una opción viable? 
- Hacer padding con 0's (tantos como el tamaño de la ventana)?

In [66]:
# TODO: code me!

### Promedio

Como su nombre indica, se calcula el centroide de los embeddings de todas las palabras circundantes. La fórmula divide cada dimensión en $2W$ ya que el número de palabras del contexto es dos veces el tamaño de la ventana:

$$\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} \frac{w_{ij}}{2W}$$

DUDA: nuevamente como manejar caso de las palabras que están al comienzo ($W$ > #palabras a la izquierda) y al final ($W$ > #palabras a la derecha)  

In [67]:
# TODO: code me!

### Decaimiento fraccional

Una tercera estrategia para construir un vector de carácteristicas en base a los embeddings de palabras contextuales está inspirada en la forma en que Word2vec combina las palabras en el contexto. Aquí, se supone que la importancia de una palabra para nuestra representación es inversamente proporcional a su distancia respecto a la palabra objetivo.

Por lo tanto, las palabras contextuales se ponderan en función de su distancia de la palabra objetivo:

$$\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} *\frac{W - \lvert I-j\rvert}{W}$$


In [68]:
# TODO: code me!

### Decaimiento exponencial

Funciona de manera similar al decaimiento fraccional, que le da más importancia al contexto cercano, pero en este caso la ponderación se realiza exponencialmente:

$$\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} * (1 - \alpha)^{\lvert \space I\space-\space j\space\rvert\space-\space1}$$

donde $\alpha = 1 - 0.1^{(W-1)^{-1}}$ es el parámetro de decaimiento. Elegimos el parámetro de tal manera que las palabras inmediatas que rodean a la palabra objetivo contribuyen 10 veces más que las últimas palabras en ambos lados de la ventana.

In [69]:
# TODO: code me!